In [1]:
from __future__ import annotations

import attrs

import vocalpy as voc

In [2]:
@attrs.define
class SequenceDataset:
    sequences: list[vocalpy.Sequence] = attrs.field()

    def to_sqlite(db_path: str):
        # TODO: write this so that we always re-create the whole thing from scratch;
        # don't bother checking whether anything is in the database because we *know* we just made it
        # that's the whole point
        pass

    def from_sqlite(db_path: str):
        pass

In [3]:
cd /Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy

/Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy


In [4]:
ls tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/

032312/ 032412/


In [5]:
data_dir = 'tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/'

In [6]:
cbins = voc.paths.from_dir(data_dir, 'cbin')

In [7]:
aud = voc.Audio(path=cbins[0])

In [8]:
audios = [voc.Audio.read(cbin) for cbin in cbins]

In [9]:
segment_params = {
    'threshold': 1500,
    'min_dur': 0.01,
    'min_silent_dur': 0.006,
}
segmenter = voc.Segmenter(segment_params=segment_params)

In [10]:
seqs = segmenter.segment(audios)

[                                        ] | 0% Completed | 131.14 us

/Users/davidnicholson/Documents/repos/coding/vocalpy/vocalpy/src/vocalpy/signal/audio.py:47: UserWarning: Values in `data` would overflow when squaring because of dtype, >i2;casting to a larger integer dtype to avoid
  warnings.warn(


[########################################] | 100% Completed | 106.46 ms


In [11]:
import sqlalchemy

In [12]:
import enum

from sqlalchemy import (
    Column,
    create_engine,
    Enum,
    Float,
    ForeignKey,
    Integer,
    String,
    Table,
)
from sqlalchemy.orm import DeclarativeBase

from sqlalchemy.orm import relationship

import vocalpy as voc

In [13]:
class Base(DeclarativeBase):
    pass

In [14]:
class Audio(Base):
    __tablename__ = 'audios'
    
    id = Column(Integer, primary_key=True)
    path = Column(String)
    
    sequences = relationship("Sequence", back_populates="audio")

In [15]:
class SegmentParams(Base):
    __tablename__ = 'segment_params'
    id = Column(Integer, primary_key=True)
    path = Column(String)
    
    sequences = relationship("Sequence", back_populates="segment_params")

In [16]:
class Sequence(Base):
    __tablename__ = 'sequences'
    
    id = Column(Integer, primary_key=True)
    audio_id = Column(Integer, ForeignKey('audios.id'))
    onset = Column(Float)
    offset = Column(Float)
    method = Column(String)  # should this be a table?
    segment_params_id = Column(Integer, ForeignKey('segment_params.id'))

    audio = relationship("Audio", back_populates="sequences")
    segment_params = relationship("SegmentParams", back_populates="sequences")
    
    units = relationship("Unit", back_populates="sequence")


In [17]:
class Unit(Base):
    __tablename__ = 'units'
    
    id = Column(Integer, primary_key=True)
    sequence_id = Column(Integer, ForeignKey('sequences.id'))
    onset = Column(Float)
    offset = Column(Float)
    
    sequence = relationship("Sequence", back_populates="units")

In [19]:
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session

In [18]:
def get_sqlite_engine(db_path=':memory:', echo=True):
     return create_engine(f'sqlite:///{db_path}', echo=echo)

def get_session(engine):
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

In [32]:
import json
import pathlib

def to_sqlite(seqs, db_name=':memory:', dst=None, echo=True):
    if dst:
        dst = pathlib.Path(dst)
        if not dst.exist() or not dst.is_dir():
            raise NotADirectory(
                f"`dst` not found or not recognized as a directory: {dst}"
            )
        url = f'sqlite:///{dst}{db_name}'
    else:
        url = f'sqlite:///{db_name}'

    engine = create_engine(url, echo=echo)

    Base.metadata.create_all(engine)

    # TODO: handle checking here -- 
    # actually make sure in Segmenter that `segment_params` can be serialized
    
    # make a set to get unique values, 
    # then make a list so we can do uniq_segment_params.index in loop below

    segment_params_map = {}
    uniq_segment_params = []
    # need to do it this way since we can't hash a dict to use `set` to find unique
    # https://stackoverflow.com/questions/11092511/list-of-unique-dictionaries
    for seq in seqs:
        if seq.segment_params not in uniq_segment_params:
            uniq_segment_params.append(segment_params)

    orm_segment_params = []
    for ind, segment_params_dict in enumerate(uniq_segment_params):
        fname = f'segment-params-{ind + 1}.json'
        if dst:
            segment_params_json_path = dst / fnane
        else:
            segment_params_json_path = pathlib.Path(fname)
        with segment_params_json_path.open('w') as fp:
            json.dump(segment_params_dict, fp)
        orm_segment_params.append(
            SegmentParams(path=str(segment_params_json_path))
        )

    with Session(engine) as session, session.begin():
        for an_orm_segment_params in orm_segment_params:
            session.add(an_orm_segment_params)
        
        for seq in seqs:
            audio = Audio(path=str(seq.audio.path))
            session.add(audio)

            # make and add sequence, referring to audio and segment params
            ind = uniq_segment_params.index(seq.segment_params)
            an_orm_segment_params = orm_segment_params[ind]

            sequence = Sequence(
                audio=audio,
                segment_params=an_orm_segment_params,
                onset=seq.onset,
                offset=seq.offset,
                method=seq.method,
            )
            session.add(sequence)

            # make and add units
            for seq_unit in seq.units:
                unit = Unit(
                    onset=seq_unit.onset,
                    offset=seq_unit.offset,
                    sequence=sequence
                )
        session.add(unit)
        # ---- implicit session.commit() when we __exit__ context + begin() from above

In [33]:
to_sqlite(seqs)

2023-05-07 10:00:26,254 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-07 10:00:26,255 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("audios")
2023-05-07 10:00:26,256 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 10:00:26,262 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("audios")
2023-05-07 10:00:26,263 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 10:00:26,265 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("segment_params")
2023-05-07 10:00:26,266 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 10:00:26,268 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("segment_params")
2023-05-07 10:00:26,269 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 10:00:26,270 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sequences")
2023-05-07 10:00:26,271 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-07 10:00:26,273 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sequences")
2023-05-07 10:00:26,276 INFO sqlalchemy.engine.Eng

/var/folders/th/rsbsynzn56zbz5vrtgv63yh00000gn/T/ipykernel_19293/355096502.py:46: SAWarning: Object of type <Unit> not in session, add operation along 'Sequence.units' will not proceed
  with Session(engine) as session, session.begin():


In [21]:
engine = get_sqlite_engine(db_path='gy6or6-032312.db')
session = get_session(engine)

# next line makes tables?
Base.metadata.create_all(engine)

2023-05-06 22:27:55,788 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-06 22:27:55,789 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("audios")
2023-05-06 22:27:55,790 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-06 22:27:55,791 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("audios")
2023-05-06 22:27:55,793 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-06 22:27:55,794 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("segment_params")
2023-05-06 22:27:55,796 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-06 22:27:55,797 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("segment_params")
2023-05-06 22:27:55,798 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-06 22:27:55,802 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sequences")
2023-05-06 22:27:55,803 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-06 22:27:55,806 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sequences")
2023-05-06 22:27:55,807 INFO sqlalchemy.engine.Eng

In [1]:
seqs[0].segment_params

NameError: name 'seqs' is not defined

In [24]:
# make segment params
# should do this in real method by applying set to all Sequence.segment_params instances
segment_params = get_or_add_segment_params(session, path='fake-segment-params.json')

# TODO: write this so that we always re-create the whole thing from scratch;
# don't bother checking whether anything is in the database because we *know* we just made it
# that's the whole point

for seq in seqs:
    # make and add audio
    audio = Audio(path=str(path))
    session.add(audio)

    # make and add sequence, referring to audio and segment params
    sequence = Sequence(
        audio=audio,
        segment_params=segment_params,
        onset=seq.onset,
        offset=seq.offset,
        method=seq.method,
    )
    session.add(sequence)
    
    # make and add units
    for seq_unit in seq.units:
        unit = Unit(
            onset=seq_unit.onset,
            offset=seq_unit.offset,
            sequence=sequence
        )
        session.add(unit)

2023-05-06 22:28:02,633 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-06 22:28:02,653 INFO sqlalchemy.engine.Engine SELECT segment_params.id, segment_params.path 
FROM segment_params 
WHERE segment_params.path = ?
2023-05-06 22:28:02,654 INFO sqlalchemy.engine.Engine [generated in 0.00115s] ('fake-segment-params.json',)
2023-05-06 22:28:02,657 INFO sqlalchemy.engine.Engine INSERT INTO segment_params (path) VALUES (?)
2023-05-06 22:28:02,658 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ('fake-segment-params.json',)
2023-05-06 22:28:02,659 INFO sqlalchemy.engine.Engine COMMIT
2023-05-06 22:28:02,662 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-06 22:28:02,664 INFO sqlalchemy.engine.Engine SELECT audios.id, audios.path 
FROM audios 
WHERE audios.path = ?
2023-05-06 22:28:02,667 INFO sqlalchemy.engine.Engine [generated in 0.00245s] ('tests/data-for-tests/source/audio_cbin_annot_notmat/gy6or6/032312/gy6or6_baseline_230312_0808.138.cbin',)
2023-05-06 22:28:02,671 

In [ ]:
@model(family=WindowedFrameClassificationModel)
class WindowedTweetyNet:
    network = nets.TweetyNet
    ...


@model(family=FrameClassificationModel)
class WholeSpectrogramTweetyNet:
    network = nets.TweetyNet
    ...